In [37]:
import tensorflow as tf
import tensorlayer as tl
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import math
import re

In [38]:
batch_size = 1# 한 iteration에 몇개의 batch를 사용할지
image_resize_w = 256
image_resize_h = 256
nstack = 2  # num of hourglass stack

net_ouput_w = 64
net_ouput_h = 64

save_freq = 20000 # model weights를 저장하는 주기
save_dir = "./model"

vis_test_dir = "./vis/test"

path_pose_joint = "/media/vision/Seagate Expansion Drive/coco2014data/original/test" # 데이터셋의 경로

test_imgs = os.listdir(path_pose_joint)
test_imgs_num = int(len(test_imgs)/3)
    
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
if not os.path.exists(vis_test_dir):
    os.makedirs(vis_test_dir)


In [39]:
def _data_aug_fn(image, joints_image, segs_image):
    h = len(image)
    w = len(image[0])

    joint = np.zeros([net_ouput_h,net_ouput_w,18], dtype=np.float32)  # 17 + background
    existing_joints = np.unique(joints_image)
    
    for existing_joint in existing_joints:
        if existing_joint != 0:
            tmp_y = np.where(joints_image == existing_joint)[0][0]
            tmp_x = np.where(joints_image == existing_joint)[1][0]
            resized_x = int(tmp_x/w * net_ouput_w)
            resized_y = int(tmp_y/h * net_ouput_h)
            joint[resized_y][resized_x][int(existing_joint/10)] = 1.0
    
    joint_bg = 1 - np.sum(joint[:,:,1:], axis=-1)
    joint_bg[joint_bg < 0] = 0
    joint[:,:,0] = joint_bg
        
    segs_image = np.squeeze(segs_image, axis=-1)
    seg = np.eye(15,dtype=np.float32)[np.int32(segs_image/10)]
    
    
    image = cv2.resize(image, (image_resize_h, image_resize_w))#, interpolation=cv2.INTER_NEAREST)
    seg = cv2.resize(seg, (net_ouput_h, net_ouput_w), interpolation=cv2.INTER_NEAREST)
    
    return image, joint, seg


In [40]:
def _map_fn(frame_name, joint_name, seg_name):
    image = tf.image.decode_jpeg(tf.read_file(frame_name), channels=3)
    joints_image = tf.image.decode_jpeg(tf.read_file(joint_name), channels=1)
    segs_image = tf.image.decode_png(tf.read_file(seg_name), channels=1)
     
    img, joint, seg  = tf.py_func(_data_aug_fn, [image, joints_image, segs_image], [tf.uint8, tf.float32, tf.float32])
    
    img = tf.image.convert_image_dtype(img, dtype=tf.float32)
    
    
    return img, joint, seg, image

    

In [41]:
def densepose_agu(directory):
    frame_name = []
    joint_name = []
    seg_name = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.jpg'):
                tmp = []
                frame_name.append(root + '/' + file)
                joint_name.append(root + '/' + file[:-4] + '_keypoint.png')
                seg_name.append(root + '/' + file[:-4] + '_seg.png')
    dataset = tf.data.Dataset.from_tensor_slices((frame_name, joint_name, seg_name))
            
    ds_test = dataset.map(_map_fn)

    ds_test = ds_test.batch(batch_size)  
    ds_test = ds_test.repeat()
    return ds_test


In [42]:
def residual(inputs, input_ch, ouput_ch, is_train):
    if input_ch != ouput_ch:
        identity = tf.layers.conv2d(inputs, ouput_ch, [1, 1], padding='same')#, activation=tf.nn.relu)
        identity = tf.nn.relu(tf.layers.batch_normalization(identity, training=is_train))

    else:
        identity = inputs

    net = tf.layers.conv2d(inputs, input_ch, [1, 1], padding='same')#, activation=tf.nn.relu)
    net = tf.nn.relu(tf.layers.batch_normalization(net, training=is_train))

    net = tf.layers.conv2d(net, input_ch, [3, 3], padding='same')#, activation=tf.nn.relu)
    net = tf.nn.relu(tf.layers.batch_normalization(net, training=is_train))

    net = tf.layers.conv2d(net, ouput_ch, [1, 1], padding='same')
    net = tf.layers.batch_normalization(net, training=is_train)
           
    net += identity
    return net

def pre_hourglass(img, is_train):
    save256 = residual(img, 3, 64, is_train) # 256 256 64
    save256 = residual(save256, 64, 256, is_train) # 256 256 256
    save256 = residual(save256, 256, 256, is_train) # 256 256 256
 
    net = tf.layers.conv2d(img, 64, [7, 7], strides = 2, padding='same')#, activation=tf.nn.relu)
    net = tf.nn.relu(tf.layers.batch_normalization(net, training=is_train))

  #  print("128x128")
  #  print(net.shape)   #128 128 64
    
    net = residual(net, 64, 128, is_train)
    
    save128 = residual(net, 128, 256, is_train) # 128 128 64
    save128 = residual(save128, 256, 256, is_train) # 128 128 256
    save128 = residual(save128, 256, 256, is_train) # 128 128 256
    
    net = tf.layers.max_pooling2d(net, [2, 2], [2, 2])
   # print("64x64") 
   # print(net.shape) # 64 64 128
    
    net = residual(net, 128, 256, is_train) # 64 64 256
    return net, save256, save128


def post_hourglass(net, save256, save128, is_train):      ## input : 64 x 64 x 256
    net = residual(net, 256, 256, is_train)
    
    net = tf.image.resize_nearest_neighbor(net, (128,128))
    net += save128
    
    net = residual(net, 256, 256, is_train)
    
    net = tf.image.resize_nearest_neighbor(net, (256,256))
    net += save256
    
    return net   
    
def hourglass(net, is_train):#, is_first = True):   ### input : 64 x 64 x 256
    net = residual(net, 256, 256, is_train)
    net = residual(net, 256, 256, is_train)

    save64 = residual(net, 256, 256, is_train) # 64 64 256
    save64 = residual(save64, 256, 256, is_train) # 64 64 256
    save64 = residual(save64, 256, 256, is_train) # 64 64 256

    
    net = tf.layers.max_pooling2d(net, [2, 2], [2, 2]) # 32 32 256
    net = residual(net, 256, 256, is_train)
    net = residual(net, 256, 256, is_train)
    net = residual(net, 256, 256, is_train)

    save32 = residual(net, 256, 256, is_train) # 32 32 256
    save32 = residual(save32, 256, 256, is_train) # 32 32 256
    save32 = residual(save32, 256, 256, is_train) # 32 32 256

    
    net = tf.layers.max_pooling2d(net, [2, 2], [2, 2]) # 16 16 256
    net = residual(net, 256, 256, is_train)
    net = residual(net, 256, 256, is_train)
    net = residual(net, 256, 256, is_train)

    save16 = residual(net, 256, 256, is_train) # 16 16 256
    save16 = residual(save16, 256, 256, is_train) # 16 16 256
    save16 = residual(save16, 256, 256, is_train) # 16 16 256

    
    net = tf.layers.max_pooling2d(net, [2, 2], [2, 2]) # 8 8 256
    net = residual(net, 256, 256, is_train)
    net = residual(net, 256, 256, is_train)
    net = residual(net, 256, 256, is_train)

    
    save8 = residual(net, 256, 256, is_train) # 8 8 256
    save8 = residual(save8, 256, 256, is_train) # 8 8 256
    save8 = residual(save8, 256, 256, is_train) # 8 8 256

    
    net = tf.layers.max_pooling2d(net, [2, 2], [2, 2]) # 4 4 256
    
    net = residual(net, 256, 256, is_train)
    net = residual(net, 256, 256, is_train)
    net = residual(net, 256, 256, is_train)
    
    
    net = tf.image.resize_nearest_neighbor(net, (8,8))
    net += save8
    
    net = residual(net, 256, 256, is_train)

    
    net = tf.image.resize_nearest_neighbor(net, (16,16))
    net += save16
    
    net = residual(net, 256, 256, is_train)

    
    net = tf.image.resize_nearest_neighbor(net, (32,32))
    net += save32

    net = residual(net, 256, 256, is_train)

    
    net = tf.image.resize_nearest_neighbor(net, (64,64))
    net += save64
    
    net = residual(net, 256, 256, is_train)
    
    inter_seg = tf.layers.conv2d(net, 15, [1, 1], padding='same')#, activation=tf.nn.relu)  ###input : 256 channel
    interpredict_seg = tf.layers.batch_normalization(inter_seg, training=is_train)
    
    net = residual(interpredict_seg, 15, 128, is_train)
    net = residual(net, 128, 256, is_train)
    net += save64
    
    inter_key = tf.layers.conv2d(net, 18, [1, 1], padding='same')#, activation=tf.nn.relu)  ###input : 256 channel
    interpredict_key = tf.layers.batch_normalization(inter_key, training=is_train)
    
    return net, interpredict_key, interpredict_seg   ## 64x64x256,  64x64x33
#######################################################
 
    

In [43]:
def SHGnet(img, numstack, is_train): ##img = [batch, w, h, 3], hourglass-ing
    with tf.variable_scope("SegNet"):
        net, save256, save128 = pre_hourglass(img, is_train)  ### 256 -> 64

        interpredicts_key = []
        interpredicts_seg = []
        for i in range(numstack):
            savenet = net
            net, interpredict_key, interpredict_seg = hourglass(net, is_train)#, False)
            interpredicts_key.append(interpredict_key)
            interpredicts_seg.append(interpredict_seg)

            inter_key = tf.layers.conv2d(interpredict_key, 256, [1, 1], padding='same')#, activation=tf.nn.relu)  ###input : 256 channel
            inter_key = tf.nn.relu(tf.layers.batch_normalization(inter_key, training=is_train))
            inter_seg = tf.layers.conv2d(interpredict_seg, 256, [1, 1], padding='same')#, activation=tf.nn.relu)  ###input : 256 channel
            inter_seg = tf.nn.relu(tf.layers.batch_normalization(inter_seg, training=is_train))
            
            net += inter_key
            net += inter_seg
            net += savenet
        

        return interpredicts_key, interpredicts_seg

In [44]:
def save_train(save_dir, input_image, joint_gt_index, seg_gt, joint_predict_index, seg_predict, step):   #batch result,img
    image = input_image[0]
    
    h = len(image)
    w = len(image[0])

    seg_gt = seg_gt[0]
    seg_pred = seg_predict[0]

    image = cv2.flip(image, 0)
    plt.clf()
    implot = plt.imshow(image)    
    for i in range(0,17):
        if joint_gt_index[i][0] != 0 and joint_gt_index[i][1] != 0:
            if i != 1 and i != 2 and i != 3 and i != 4:
                plt.scatter(joint_gt_index[i][1]/64 * w, (64 - joint_gt_index[i][0])/64 * h, s = 4, color = 'blue', alpha = 0.8)
                plt.scatter(joint_predict_index[i][1]/64 * w, (64 - joint_predict_index[i][0])/64 * h, s = 4, color = 'red', alpha = 0.8)
    plt.xlim(0, w)
    plt.ylim(0, h) 
#########################    
    nose_x_gt = joint_gt_index[0][1]/64 * w
    nose_y_gt = (64-joint_gt_index[0][0])/64 * h

    left_shoulder_x_gt = joint_gt_index[5][1]/64 *w
    left_shoulder_y_gt = (64-joint_gt_index[5][0])/64 * h
    
    right_shoulder_x_gt = joint_gt_index[6][1]/64 * w
    right_shoulder_y_gt = (64-joint_gt_index[6][0])/64 * h
    
    neck_x_gt = (left_shoulder_x_gt + right_shoulder_x_gt)/2
    neck_y_gt = (left_shoulder_y_gt + right_shoulder_y_gt)/2
    
    left_elbow_x_gt = joint_gt_index[7][1]/64 * w
    left_elbow_y_gt = (64-joint_gt_index[7][0])/64 * h
    
    right_elbow_x_gt = joint_gt_index[8][1]/64 * w
    right_elbow_y_gt = (64-joint_gt_index[8][0])/64 * h
    
    left_wrist_x_gt = joint_gt_index[9][1]/64 * w
    left_wrist_y_gt = (64-joint_gt_index[9][0])/64 * h
    
    right_wrist_x_gt = joint_gt_index[10][1]/64 * w
    right_wrist_y_gt = (64-joint_gt_index[10][0])/64 * h
    
    left_hip_x_gt = joint_gt_index[11][1]/64 * w
    left_hip_y_gt = (64-joint_gt_index[11][0])/64 * h
    
    right_hip_x_gt = joint_gt_index[12][1]/64 * w
    right_hip_y_gt = (64-joint_gt_index[12][0])/64 * h
    
    hip_x_gt = (left_hip_x_gt + right_hip_x_gt)/2
    hip_y_gt = (left_hip_y_gt + right_hip_y_gt)/2

    left_knee_x_gt = joint_gt_index[13][1]/64 * w
    left_knee_y_gt = (64-joint_gt_index[13][0])/64 * h
    
    right_knee_x_gt = joint_gt_index[14][1]/64 * w
    right_knee_y_gt = (64-joint_gt_index[14][0])/64 * h
    
    left_ankle_x_gt = joint_gt_index[15][1]/64 * w
    left_ankle_y_gt = (64-joint_gt_index[15][0])/64 * h
    
    right_ankle_x_gt = joint_gt_index[16][1]/64 * w
    right_ankle_y_gt = (64-joint_gt_index[16][0])/64 * h
    ################################3
    nose_x_dt = joint_predict_index[0][1]/64 * w
    nose_y_dt = (64-joint_predict_index[0][0])/64 * h

    left_shoulder_x_dt = joint_predict_index[5][1]/64 * w
    left_shoulder_y_dt = (64-joint_predict_index[5][0])/64 * h
    
    right_shoulder_x_dt = joint_predict_index[6][1]/64 * w
    right_shoulder_y_dt = (64-joint_predict_index[6][0])/64 * h
    
    neck_x_dt = (left_shoulder_x_dt + right_shoulder_x_dt)/2
    neck_y_dt = (left_shoulder_y_dt + right_shoulder_y_dt)/2
    
    left_elbow_x_dt = joint_predict_index[7][1]/64 * w
    left_elbow_y_dt = (64-joint_predict_index[7][0])/64 * h
    
    right_elbow_x_dt = joint_predict_index[8][1]/64 * w
    right_elbow_y_dt = (64-joint_predict_index[8][0])/64 * h
    
    left_wrist_x_dt = joint_predict_index[9][1]/64 * w
    left_wrist_y_dt = (64-joint_predict_index[9][0])/64 * h
    
    right_wrist_x_dt = joint_predict_index[10][1]/64 * w
    right_wrist_y_dt = (64-joint_predict_index[10][0])/64 * h
    
    left_hip_x_dt = joint_predict_index[11][1]/64 * w
    left_hip_y_dt = (64-joint_predict_index[11][0])/64 * h
    
    right_hip_x_dt = joint_predict_index[12][1]/64 * w
    right_hip_y_dt = (64-joint_predict_index[12][0])/64 * h
    
    hip_x_dt = (left_hip_x_dt + right_hip_x_dt)/2
    hip_y_dt = (left_hip_y_dt + right_hip_y_dt)/2

    left_knee_x_dt = joint_predict_index[13][1]/64 * w
    left_knee_y_dt = (64-joint_predict_index[13][0])/64 * h
    
    right_knee_x_dt = joint_predict_index[14][1]/64 * w
    right_knee_y_dt = (64-joint_predict_index[14][0])/64 * h
    
    left_ankle_x_dt = joint_predict_index[15][1]/64 * w
    left_ankle_y_dt = (64-joint_predict_index[15][0])/64 * h
    
    right_ankle_x_dt = joint_predict_index[16][1]/64 * w
    right_ankle_y_dt = (64-joint_predict_index[16][0])/64 * h
    
    
    
    
 #   nose, neck
    if (neck_x_gt != 0 and neck_y_gt != 0) and (nose_x_gt != 0 and nose_y_gt != 0):
        x_gt, y_gt = [nose_x_gt, neck_x_gt], [nose_y_gt, neck_y_gt]
        x_dt, y_dt = [nose_x_dt, neck_x_dt], [nose_y_dt, neck_y_dt]
        plt.plot(x_gt, y_gt, color = 'blue')    
        plt.plot(x_dt, y_dt, color = 'red')   
        
#   left,right shoulder
    if (left_shoulder_x_gt != 0 and left_shoulder_y_gt != 0) and (right_shoulder_x_gt != 0 and right_shoulder_y_gt != 0):
        x_gt, y_gt = [left_shoulder_x_gt, right_shoulder_x_gt], [left_shoulder_y_gt, right_shoulder_y_gt]
        x_dt, y_dt = [left_shoulder_x_dt, right_shoulder_x_dt], [left_shoulder_y_dt, right_shoulder_y_dt]
        plt.plot(x_gt, y_gt, color = 'blue')    
        plt.plot(x_dt, y_dt, color = 'red')         
    
#   left shoulder, elbow
    if (left_shoulder_x_gt != 0 and left_shoulder_y_gt != 0) and (left_elbow_x_gt != 0 and left_elbow_y_gt != 0):
        x_gt, y_gt = [left_shoulder_x_gt, left_elbow_x_gt], [left_shoulder_y_gt, left_elbow_y_gt]
        x_dt, y_dt = [left_shoulder_x_dt, left_elbow_x_dt], [left_shoulder_y_dt, left_elbow_y_dt]
        plt.plot(x_gt, y_gt, color = 'blue')    
        plt.plot(x_dt, y_dt, color = 'red')   
        
#   right shoulder, elbow
    if (right_shoulder_x_gt != 0 and right_shoulder_y_gt != 0) and (right_elbow_x_gt != 0 and right_elbow_y_gt != 0):
        x_gt, y_gt = [right_shoulder_x_gt, right_elbow_x_gt], [right_shoulder_y_gt, right_elbow_y_gt]
        x_dt, y_dt = [right_shoulder_x_dt, right_elbow_x_dt], [right_shoulder_y_dt, right_elbow_y_dt]
        plt.plot(x_gt, y_gt, color = 'blue')    
        plt.plot(x_dt, y_dt, color = 'red')   
        
#   left elbow, wrist
    if (left_elbow_x_gt != 0 and left_elbow_y_gt != 0) and (left_wrist_x_gt != 0 and left_wrist_y_gt != 0):
        x_gt, y_gt = [left_elbow_x_gt, left_wrist_x_gt], [left_elbow_y_gt, left_wrist_y_gt]
        x_dt, y_dt = [left_elbow_x_dt, left_wrist_x_dt], [left_elbow_y_dt, left_wrist_y_dt]
        plt.plot(x_gt, y_gt, color = 'blue')    
        plt.plot(x_dt, y_dt, color = 'red')   
        
#   right elbow, wrist
    if (right_elbow_x_gt != 0 and right_elbow_y_gt != 0) and (right_wrist_x_gt != 0 and right_wrist_y_gt != 0):
        x_gt, y_gt = [right_elbow_x_gt, right_wrist_x_gt], [right_elbow_y_gt, right_wrist_y_gt]
        x_dt, y_dt = [right_elbow_x_dt, right_wrist_x_dt], [right_elbow_y_dt, right_wrist_y_dt]
        plt.plot(x_gt, y_gt, color = 'blue')    
        plt.plot(x_dt, y_dt, color = 'red')  
        
#   neck, hip
    if (neck_x_gt != 0 and neck_y_gt != 0) and (hip_x_gt != 0 and hip_y_gt != 0):
        x_gt, y_gt = [neck_x_gt, hip_x_gt], [neck_y_gt, hip_y_gt]
        x_dt, y_dt = [neck_x_dt, hip_x_dt], [neck_y_dt, hip_y_dt]
        plt.plot(x_gt, y_gt, color = 'blue')    
        plt.plot(x_dt, y_dt, color = 'red')          
        
#   left, right hip
    if (left_hip_x_gt != 0 and left_hip_y_gt != 0) and (right_hip_x_gt != 0 and right_hip_y_gt != 0):
        x_gt, y_gt = [left_hip_x_gt, right_hip_x_gt], [left_hip_y_gt, right_hip_y_gt]
        x_dt, y_dt = [left_hip_x_dt, right_hip_x_dt], [left_hip_y_dt, right_hip_y_dt]
        plt.plot(x_gt, y_gt, color = 'blue')    
        plt.plot(x_dt, y_dt, color = 'red')          

#   left hip, knee
    if (left_hip_x_gt != 0 and left_hip_y_gt != 0) and (left_knee_x_gt != 0 and left_knee_y_gt != 0):
        x_gt, y_gt = [left_hip_x_gt, left_knee_x_gt], [left_hip_y_gt, left_knee_y_gt]
        x_dt, y_dt = [left_hip_x_dt, left_knee_x_dt], [left_hip_y_dt, left_knee_y_dt]
        plt.plot(x_gt, y_gt, color = 'blue')    
        plt.plot(x_dt, y_dt, color = 'red')      
        
#   left knee, ankle
    if (left_knee_x_gt != 0 and left_knee_y_gt != 0) and (left_ankle_x_gt != 0 and left_ankle_y_gt != 0):
        x_gt, y_gt = [left_knee_x_gt, left_ankle_x_gt], [left_knee_y_gt, left_ankle_y_gt]
        x_dt, y_dt = [left_knee_x_dt, left_ankle_x_dt], [left_knee_y_dt, left_ankle_y_dt]
        plt.plot(x_gt, y_gt, color = 'blue')    
        plt.plot(x_dt, y_dt, color = 'red')      

#   right hip, knee
    if (right_hip_x_gt != 0 and right_hip_y_gt != 0) and (right_knee_x_gt != 0 and right_knee_y_gt != 0):
        x_gt, y_gt = [right_hip_x_gt, right_knee_x_gt], [right_hip_y_gt, right_knee_y_gt]
        x_dt, y_dt = [right_hip_x_dt, right_knee_x_dt], [right_hip_y_dt, right_knee_y_dt]
        plt.plot(x_gt, y_gt, color = 'blue')    
        plt.plot(x_dt, y_dt, color = 'red')      
        
#   right knee, ankle
    if (right_knee_x_gt != 0 and right_knee_y_gt != 0) and (right_ankle_x_gt != 0 and right_ankle_y_gt != 0):
        x_gt, y_gt = [right_knee_x_gt, right_ankle_x_gt], [right_knee_y_gt, right_ankle_y_gt]
        x_dt, y_dt = [right_knee_x_dt, right_ankle_x_dt], [right_knee_y_dt, right_ankle_y_dt]
        plt.plot(x_gt, y_gt, color = 'blue')    
        plt.plot(x_dt, y_dt, color = 'red')      
 #######################################3       
        
        
        
    plt.savefig(save_dir + '/img' + str(step) +'_joint.png', bbox_inches='tight', pad_inches=0)
    
    
#######    
    
    
    seg_gt = np.argmax(seg_gt,-1)

    
    seg_pred = np.argmax(seg_pred,-1)
    
    image = cv2.flip(image, 0)
    plt.imsave(save_dir + '/img' + str(step) +'_.png', image)

 #   plt.imsave(save_dir + '/img' + str(step) +'_joint_gt.png', joint_gt)
 #   plt.imsave(save_dir + '/img' + str(step) +'_joint_pred.png', joint_pred)
    plt.imsave(save_dir + '/img' + str(step) +'_seg_gt.png', seg_gt)
    plt.imsave(save_dir + '/img' + str(step) +'_seg_pred.png', seg_pred)
    

  #  tmp = np.amax(tmp[:,:,1:],-1)
  #  tmp *= 255
  #  plt.imsave(save_dir + '/img' + str(step) +'_joint_amax.png', tmp)

In [45]:
def euclidean_cal(point1, point2):
    point1_x = point1[0]
    point1_y = point1[1]
    point2_x = point2[0]
    point2_y = point2[1]
    
    a = point1_x - point2_x
    b = point1_y - point2_y
    aa = a*a
    bb = b*b
    cc = aa + bb
    return np.sqrt(cc)

def PCK_eval(label, pred, pck_rate):
    label = label[:,:,:,1:]
    pred = pred[:,:,:,1:]
    
    keypoint_num = 0
    anwser_num = 0
    
    rate = pck_rate

    maxindex_list_gt = []
    maxindex_list_dt = []
    
    for i in range(0,17):
        yAndx_gt = []

        y_gt, x_gt = np.where(label[0,:,:,i] == np.max(label[0,:,:,i])) #(array([], dtype=int64),)
        yAndx_gt.append(y_gt[0])
        yAndx_gt.append(x_gt[0])
        maxindex_list_gt.append(yAndx_gt)

        yAndx_dt = []

        y_dt, x_dt = np.where(pred[0,:,:,i] == np.max(pred[0,:,:,i]))
        yAndx_dt.append(y_dt[0])
        yAndx_dt.append(x_dt[0])
        maxindex_list_dt.append(yAndx_dt)

    torso = euclidean_cal(maxindex_list_gt[5], maxindex_list_gt[12])
    
    for i in range(0,17):
        if i != 1 or i != 2 or i != 3 or i != 4: # without left,right_eye, ear  
            if maxindex_list_gt[i] == [0,0]:
                continue
            else:
                keypoint_num += 1
                distance = euclidean_cal(maxindex_list_gt[i], maxindex_list_dt[i])
                if distance <= torso * rate:
                    anwser_num += 1
    
    return anwser_num/keypoint_num, maxindex_list_gt, maxindex_list_dt

In [46]:
if __name__ == '__main__':
    tf.reset_default_graph()
    ds_test = densepose_agu(path_pose_joint)
    
    iterator = ds_test.make_one_shot_iterator()
    one_element = iterator.get_next()    

    img_ph = tf.placeholder(tf.float32, [batch_size, image_resize_w, image_resize_h, 3])
    joint_gt_ph = tf.placeholder(tf.float32, [batch_size, net_ouput_w, net_ouput_h, 18])
    seg_gt_ph = tf.placeholder(tf.float32, [batch_size, net_ouput_w, net_ouput_h, 15])
    
    is_train = tf.placeholder(tf.bool, shape=())
    
    predictions_key, predictions_seg = SHGnet(img_ph, nstack, is_train)
        
    joint_predict = predictions_key[nstack-1]
    seg_predict = predictions_seg[nstack-1]
    
    saver = tf.train.Saver()
    
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

In [47]:
def IOU_eval(gt_batch, dt_batch):
    gt = gt_batch[0]
    dt = dt_batch[0]

    gt = np.argmax(gt,-1)

    
    dt = np.argmax(dt,-1)

    count = 0
    iou = 0
    
    for i in range(1,15):  # 1 ~ 14
        if i == 1 or i ==14:
            a = np.zeros([64,64], dtype=np.int32)
            b = np.zeros([64,64], dtype=np.int32)
            for x in range(64):
                for y in range(64):
                    if gt[x][y] == i or dt[x][y] == i:
                        a[x][y] = 1
                    if gt[x][y] == i and dt[x][y] == i:
                        b[x][y] = 1
            if np.sum(a) != 0:            
                iou += np.sum(b)/np.sum(a)
                count += 1
        
    return iou/count
                

In [48]:
# def IOU_eval(gt_batch, dt_batch):
#     gt = gt_batch[0]
#     dt = dt_batch[0]

#     gt = np.argmax(gt,-1)

    
#     dt = np.argmax(dt,-1)

#     count = 0
#     U = 0
#     N = 0
#     for i in range(1,15):  # 1 ~ 14
#         for x in range(64):
#             for y in range(64):
#                 if gt[x][y] == i or dt[x][y] == i:
#                     U += 1
#                 if gt[x][y] == i and dt[x][y] == i:
#                     N += 1
        
#     return N/U
                

In [49]:
sess = tf.Session()


for pck in range(5,6):
    pck_rate = pck * 0.1
    print('pck@', pck_rate)
    for i in range(180000,180001,20000):
        sess.run(tf.global_variables_initializer())
        step = 0

        load = save_dir + "/model-" + str(i)

        saver.restore(sess, load)
      #  saver.restore(sess, tf.train.latest_checkpoint(save_dir))

        total_pck = 0
        total_iou = 0
        count = 0
        while True:
            GT_imgNseg = sess.run(one_element)
            joint_predicted, seg_predicted = sess.run([joint_predict, seg_predict], feed_dict={img_ph:GT_imgNseg[0], joint_gt_ph:GT_imgNseg[1], seg_gt_ph:GT_imgNseg[2], is_train:False})

            if np.max(GT_imgNseg[1][:,:,:,6]) == 1 and np.max(GT_imgNseg[1][:,:,:,13]) == 1:
                count += 1
                point, gts, dts = PCK_eval(GT_imgNseg[1], joint_predicted, pck_rate)
                
                
                iou = IOU_eval(GT_imgNseg[2], seg_predicted)
                
                
                total_pck += point
                total_iou += iou
             #   save_train(vis_test_dir, GT_imgNseg[3], gts, GT_imgNseg[2], dts, seg_predicted, step)


            step += 1
            if step == test_imgs_num:
                break
     #   print(total_pck/count)
        print(total_iou/count)


pck@ 0.5
INFO:tensorflow:Restoring parameters from ./model/model-180000
0.5304713086303564
